In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd F:\nvb\KB-InfoBot

F:\nvb\KB-InfoBot


In [3]:
from deep_dialog.dialog_system import MovieDict, DictReader, Database
from deep_dialog.objects import SlotReader
from deep_dialog.agents.agent_act_rule import AgentActRule
from deep_dialog.usersims import RuleSimulator
from deep_dialog.usersims.template_nlg import TemplateNLG
from deep_dialog.dialog_system.dialog_manager import DialogManager

In [4]:
params = {}
params['act_set'] = r'F:\nvb\KB-InfoBot\data\dia_acts.txt'
params['dataset'] = 'imdb_S'
params['tr'] = 1.0
params['ts'] = 1.0
params['max_req'] = 1
params['frac'] = 0.5
params['upd'] = 10
params['template_path'] = r'F:\nvb\KB-InfoBot\data\templates.j'

max_turn = 20
err_prob = .5
dk_prob = .5
sub_prob = .05
nlg_temp = 1
max_first_turn = 5


slot_path = r'F:\nvb\KB-InfoBot\data\imdb-S\slot_set.txt'
dict_path = r'F:/nvb/KB-InfoBot/data/imdb-S/dicts_.json'
db_full_path = r'F:\nvb\KB-InfoBot\data\imdb-S\db.txt'
db_inc_path = r'F:\nvb\KB-InfoBot\data\imdb-S\incomplete_db_0.20.txt'

In [5]:
act_set = DictReader()
act_set.load_dict_from_file(params['act_set'])

In [6]:
slot_set = SlotReader(slot_path)

In [7]:
movie_kb = MovieDict(dict_path)

In [8]:
db_full = Database(db_full_path, movie_kb, name=params['dataset'])
db_inc = Database(db_inc_path, movie_kb, name='incomplete_db_0.20')

In [9]:
db_matches = [m[3] for m in db_full.tuples]
db_matches[:5]

['r', 'pg-13', 'pg-13', 'pg', 'pg-13']

In [10]:
nlg = TemplateNLG(params['template_path'])

In [11]:
user_sim = RuleSimulator(movie_dict=movie_kb, act_set=act_set, slot_set=slot_set, 
                         start_set=None, max_turn=max_turn, nlg=nlg, err_prob=err_prob, 
                         db=db_full, dk_prob=(1.-dk_prob), sub_prob=sub_prob, 
                         max_first_turn=max_first_turn)

In [33]:
ua = user_sim.initialize_episode()
ua 

У искомой сущности есть слоты (в неполной БД) ['genre', 'director', 'release_year']
Пользователь уже назвал (?): {'release_year': '2003'}


{'diaact': 'request',
 'turn': 0,
 'inform_slots': {'release_year': '2003'},
 'request_slots': {'movie': 'UNK'},
 'prev_diaact': 'UNK',
 'inform_slots_noisy': {'release_year': '2003'},
 'nl_sentence': 'Which movie has 2003 as release_year?',
 'episode_over': False,
 'reward': 0}

In [12]:
agent = AgentActRule(movie_kb, act_set, slot_set, db_inc,
            upd=params['upd'], tr=params['tr'], ts=params['ts'], 
            frac=params['frac'], max_req=params['max_req'])

In [35]:
agent.initialize_episode()
agent.state

{'diaact': 'UNK', 'inform_slots': {}, 'turn': 0}

In [36]:
nxt = agent.next(ua)
agent.state

query for lookup [None, None, None, None, None, '2003']
Нашло 63 строк
в запросе участвовали не все слоты и нашелся не один ответ
Энтропии:  {'actor': 1.4947969915434802, 'critic_rating': 3.2487639119923792, 'genre': 3.309520095059927, 'mpaa_rating': 1.9891117711637363, 'director': 2.3113992074527006}


{'diaact': 'UNK', 'inform_slots': {'release_year': '2003'}, 'turn': 0}

In [37]:
nxt

{'diaact': 'request',
 'request_slots': {'genre': 'UNK'},
 'target': [],
 'posterior': array([0.01587302, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.01587302, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.01587302,
        0.        , 0.        , 0.        , 0.01587302, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01587302, 0.        , 0.01587302, 0.        , 0.        ,
        0.        , 0.01587302, 0.        , 0.01587302, 0.01587302,
        0.        , 0.        , 0.        , 0.01587302, 0.        ,
        0.        , 0.        , 0.01587302, 0.01587302, 0.01587302,
        0.        , 0.01587302, 0.01587302, 0.        , 0.        ,
        0.        , 0.01587302

In [38]:
ua, episode_over, reward = user_sim.next(nxt)

In [39]:
ua

{'diaact': 'inform',
 'turn': 1,
 'inform_slots': {'genre': 'thriller science fiction'},
 'request_slots': {},
 'prev_diaact': 'request',
 'inform_slots_noisy': {'genre': 'science fiction'},
 'nl_sentence': 'science fiction genre',
 'episode_over': False,
 'reward': -0.1}

In [40]:
nxt = agent.next(ua)
agent.state

query for lookup [None, None, 'thriller science fiction', None, None, '2003']
Нашло 19 строк
в запросе участвовали не все слоты и нашелся не один ответ
Энтропии:  {'actor': 1.5810309627563695, 'critic_rating': 2.860320777833044, 'mpaa_rating': 1.9312952802328747, 'director': 2.214812224859865}


{'diaact': 'UNK',
 'inform_slots': {'release_year': '2003', 'genre': 'thriller science fiction'},
 'turn': 0}

In [41]:
nxt

{'diaact': 'request',
 'request_slots': {'critic_rating': 'UNK'},
 'target': [],
 'posterior': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05263158, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.05263158,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05263158, 0.05263158, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.

In [42]:
ua, episode_over, reward = user_sim.next(nxt)

In [43]:
ua

{'diaact': 'inform',
 'turn': 2,
 'inform_slots': {'critic_rating': None},
 'request_slots': {},
 'prev_diaact': 'inform',
 'inform_slots_noisy': {'critic_rating': None},
 'nl_sentence': 'I dont know',
 'episode_over': False,
 'reward': -0.1}

In [44]:
nxt = agent.next(ua)
agent.state

query for lookup [None, None, 'thriller science fiction', None, None, '2003']
Нашло 19 строк
в запросе участвовали не все слоты и нашелся не один ответ
Энтропии:  {'actor': 1.5810309627563695, 'mpaa_rating': 1.9312952802328747, 'director': 2.214812224859865}


{'diaact': 'UNK',
 'inform_slots': {'release_year': '2003',
  'genre': 'thriller science fiction',
  'critic_rating': None},
 'turn': 0}

In [40]:
ua, episode_over, reward = user_sim.next(nxt)

In [41]:
ua

{'diaact': 'inform',
 'turn': 3,
 'inform_slots': {'actor': None},
 'request_slots': {},
 'prev_diaact': 'inform',
 'inform_slots_noisy': {'actor': None},
 'nl_sentence': 'I cannot remember',
 'episode_over': False,
 'reward': -0.1}

In [43]:
nxt = agent.next(ua)
agent.state

query for lookup [None, '4.5/5', 'sci-fi', None, 'zack snyder', None]
Нашло 15 строк
если в запросе участвовали все слоты либо нашелся 1 вариант => вернуть ответ


{'diaact': 'UNK',
 'inform_slots': {'critic_rating': '4.5/5',
  'genre': 'sci-fi',
  'director': 'zack snyder',
  'release_year': None,
  'mpaa_rating': None,
  'actor': None},
 'turn': 0}

In [44]:
nxt

{'diaact': 'inform',
 'request_slots': {},
 'target': [275,
  93,
  24,
  58,
  113,
  160,
  178,
  55,
  202,
  13,
  151,
  54,
  173,
  66,
  225,
  20,
  153,
  101,
  157,
  194,
  51,
  108,
  252,
  162,
  80,
  164,
  40,
  172,
  106,
  205,
  170,
  245,
  152,
  87,
  26,
  231,
  247,
  229,
  126,
  256,
  136,
  121,
  105,
  226,
  215,
  73,
  127,
  95,
  214,
  14,
  150,
  64,
  120,
  25,
  155,
  15,
  264,
  56,
  272,
  81,
  90,
  237,
  213,
  161,
  189,
  50,
  111,
  109,
  68,
  220,
  94,
  174,
  221,
  75,
  53,
  22,
  142,
  44,
  254,
  179,
  185,
  84,
  154,
  62,
  177,
  65,
  19,
  52,
  199,
  7,
  171,
  259,
  273,
  147,
  258,
  159,
  38,
  197,
  183,
  243,
  261,
  211,
  91,
  16,
  146,
  207,
  233,
  255,
  204,
  116,
  149,
  71,
  131,
  112,
  224,
  274,
  168,
  82,
  246,
  37,
  57,
  208,
  45,
  88,
  32,
  114,
  242,
  181,
  263,
  148,
  234,
  271,
  167,
  4,
  70,
  248,
  124,
  48,
  196,
  130,
  141,
  1,
  129

In [13]:
import numpy as np
import time

N = 1 # количество симуляций


user_sim = RuleSimulator(movie_dict=movie_kb, act_set=act_set, slot_set=slot_set, 
                         start_set=None, max_turn=max_turn, nlg=nlg, err_prob=err_prob, 
                         db=db_full, dk_prob=(1.-dk_prob), sub_prob=sub_prob, 
                         max_first_turn=max_first_turn)

agent = AgentActRule(movie_kb, act_set, slot_set, db_inc,
            upd=params['upd'], tr=params['tr'], ts=params['ts'], 
            frac=params['frac'], max_req=params['max_req'])

dialog_manager = DialogManager(agent, user_sim, db_full, db_inc, movie_kb, verbose=True)

In [18]:
all_rewards = np.zeros((N,))
all_success = np.zeros((N,))
all_turns = np.zeros((N,))
tst = time.time()

for i in range(N):
    current_reward = 0
    current_success = False
    ua = dialog_manager.initialize_episode()
    utt = ua['nl_sentence']
#     print('UTT: ', utt)
    t = 0
    while(True):
        t += 1
        episode_over, reward, ua, sa = dialog_manager.next_turn()
        utt = ua['nl_sentence']
        print(utt)
        current_reward += reward
        if episode_over:
            if reward > 0:
                print ("Successful Dialog! Total reward = {}".format(current_reward))
                current_success = True
            else:
                print ("Failed Dialog! Total reward = {}".format(current_reward))
            break
    all_rewards[i] = current_reward
    all_success[i] = 1 if current_success else 0
    all_turns[i] = t

У искомой сущности есть слоты (в неполной БД) ['critic_rating', 'genre', 'director']
Пользователь уже назвал (?): {'director': 'christopher nolan', 'critic_rating': '5', 'genre': 'fantasy'}
User target =  movie:heat, actor:ryan reynolds, critic_rating:5, genre:fantasy, mpaa_rating:pg-13, director:christopher nolan, release_year:2008
User information =  critic_rating:5, genre:fantasy, director:christopher nolan 

Turn 0 user action: request 	 inform slots: {'director': 'christopher nolan', 'critic_rating': '5', 'genre': 'fantasy'}
Utterance: Can you please tell me a movie whose genre is fantasy, director is christopher and critic_rating is 4? 

query for lookup [None, '5', 'fantasy', None, 'christopher nolan', None]
Нашло 13 строк
в запросе участвовали не все слоты и нашелся не один ответ
Энтропии:  {'actor': 1.4572659136233868, 'mpaa_rating': 1.4604846813131114, 'release_year': 3.026986833359287}
Turn 1 sys action: request, request slots: {'release_year': 'UNK'}

I am not sure
Turn 1 u

In [15]:
syss

[{'diaact': 'request',
  'request_slots': {'release_year': 'UNK'},
  'target': [],
  'posterior': array([0.0075188, 0.       , 0.       , 0.       , 0.       , 0.0075188,
         0.       , 0.       , 0.0075188, 0.       , 0.       , 0.0075188,
         0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
         0.0075188, 0.       , 0.       , 0.0075188, 0.       , 0.0075188,
         0.0075188, 0.0075188, 0.0075188, 0.       , 0.0075188, 0.0075188,
         0.0075188, 0.       , 0.       , 0.0075188, 0.0075188, 0.       ,
         0.0075188, 0.       , 0.0075188, 0.0075188, 0.       , 0.0075188,
         0.       , 0.0075188, 0.0075188, 0.       , 0.       , 0.       ,
         0.0075188, 0.       , 0.0075188, 0.       , 0.0075188, 0.0075188,
         0.       , 0.       , 0.       , 0.       , 0.0075188, 0.0075188,
         0.       , 0.0075188, 0.0075188, 0.0075188, 0.       , 0.0075188,
         0.0075188, 0.0075188, 0.       , 0.       , 0.0075188, 0.0075188,
    

In [47]:
ua, episode_over, reward = user_sim.next(nxt)

Просил 275, нашли на позиции 0


In [42]:
dialog_manager = DialogManager(agent, user_sim, db_full, db_inc, movie_kb, verbose=True)

In [44]:
ua = dialog_manager.initialize_episode()
ua

Пользователь знает ['genre', 'mpaa_rating', 'director']
Симулятор знает {'mpaa_rating': 'r', 'director': 'quentin tarantino'}
User target =  movie:hachi: a dog's tale, actor:ryan reynolds, critic_rating:4.5/5, genre:sci-fi, mpaa_rating:r, director:quentin tarantino, release_year:2004
User information =  genre:sci-fi, mpaa_rating:r, director:quentin tarantino 



{'diaact': 'request',
 'turn': 0,
 'inform_slots': {'mpaa_rating': 'r', 'director': 'quentin tarantino'},
 'request_slots': {'movie': 'UNK'},
 'prev_diaact': 'UNK',
 'inform_slots_noisy': {'mpaa_rating': 'r', 'director': 'christopher'},
 'nl_sentence': 'Can you please tell me a movie whose director is christopher and mpaa_rating is r?',
 'episode_over': False,
 'reward': 0}